In [1]:
from google.colab import files
import pandas as pd
import plotly.express as px

uploaded = files.upload()
df = pd.read_csv(next(iter(uploaded)))

Saving basededatosminutos.csv to basededatosminutos.csv


In [7]:
print("Columnas detectadas:", list(df.columns))
print("Primeras filas:")
print(df.head().to_string(index=False))

lower_cols = {c.lower(): c for c in df.columns}
candidates_continent = ['continent','continente','region','region_continente','region_continental']
candidates_country = ['country','pais','país','nacion','nación','nation']
candidates_minutes = ['minutes','minutes_played','minutos','min_played','min_play','mins','minutes_played','minutes_played_per_game', 'pro. de minutos'] # Added 'pro. de minutos'

def find_column(candidates, lower_cols):
    for cand in candidates:
        if cand in lower_cols:
            return lower_cols[cand]
    return None

col_continent = find_column(candidates_continent, lower_cols)
col_country = find_column(candidates_country, lower_cols)
col_minutes = find_column(candidates_minutes, lower_cols)

if col_continent is None or col_country is None or col_minutes is None:
    missing = []
    if col_continent is None: missing.append("continente")
    if col_country is None: missing.append("país")
    if col_minutes is None: missing.append("minutos jugados")
    raise ValueError("No pude detectar todas las columnas necesarias. Faltan: " + ", ".join(missing) +
                     ".\nRevisa los nombres de las columnas en el CSV o edita las variables para indicar los nombres exactos.")

df = df.rename(columns={col_continent: 'continent', col_country: 'country', col_minutes: 'minutes'})
df['minutes'] = pd.to_numeric(df['minutes'], errors='coerce')
df = df.dropna(subset=['continent','country','minutes'])

continent_mean_rows = df.groupby('continent', as_index=False)['minutes'].mean().rename(columns={'minutes':'mean_minutes_by_rows'})

country_mean = df.groupby(['continent','country'], as_index=False)['minutes'].mean().rename(columns={'minutes':'mean_minutes_by_country'})
continent_mean_countries = country_mean.groupby('continent', as_index=False)['mean_minutes_by_country'].mean().rename(columns={'mean_minutes_by_country':'mean_minutes_by_country_mean'})

summary = continent_mean_rows.merge(continent_mean_countries, on='continent')
summary['mean_minutes_by_rows'] = summary['mean_minutes_by_rows'].round(2)
summary['mean_minutes_by_country_mean'] = summary['mean_minutes_by_country_mean'].round(2)

print("\nResumen (ordenado por promedio ponderado - media de filas):")
print(summary.sort_values('mean_minutes_by_rows', ascending=False).to_string(index=False))

hover_texts = {}
for cont, group in country_mean.groupby('continent'):
    rows = group.sort_values('mean_minutes_by_country', ascending=False)
    lines = [f"{r['country']}: {r['mean_minutes_by_country']:.2f} min" for _, r in rows.iterrows()]
    hover_texts[cont] = "<br>".join(lines)

plot_df = summary.sort_values('mean_minutes_by_rows', ascending=True)

import plotly.graph_objects as go
import plotly.express as px

colors = px.colors.qualitative.Plotly
color_map = {continent: colors[i % len(colors)] for i, continent in enumerate(plot_df['continent'])}

fig = go.Figure()
fig.add_trace(go.Bar(
    x=plot_df['mean_minutes_by_rows'],
    y=plot_df['continent'],
    orientation='h',
    hovertemplate = (
        "<b>%{y}</b><br>"
        "Promedio (media de países): %{customdata[0]:.2f} min<br><br>" # Removed 'Promedio (todas las filas)'
        "Países (promedio por país):<br>%{customdata[1]}<extra></extra>"
    ),
    marker_color=[color_map[c] for c in plot_df['continent']], # Assign different colors to each bar
    customdata = list(zip(plot_df['mean_minutes_by_country_mean'], plot_df['continent'].map(hover_texts)))
))

fig.update_layout(
    title="Promedio de minutos jugados por continente",
    xaxis_title="Minutos (promedio)",
    yaxis_title="Continente",
    margin=dict(l=140, r=20, t=60, b=60),
    height=400 + 40 * len(plot_df)
)

Columnas detectadas: ['country', 'minutes', 'continent', 'Pro. de edad', 'Goles a Favor', 'Goles en Contra', 'Dif. de gol', 'Fase Llegada']
Primeras filas:
      country  minutes continent  Pro. de edad  Goles a Favor  Goles en Contra  Dif. de gol     Fase Llegada
        Chile   674.19   America         19.52              3                5           -2 Octavos de Final
        Japón   813.65      Asia         19.42              7                0            7 Octavos de Final
Nueva Zelanda   911.95   Oceanía         18.90              3                6           -3   Fase de Grupos
       Egipto   236.08    África         19.52              3                5           -2   Fase de Grupos
Corea del Sur   852.95      Asia         19.71              3                3            0 Octavos de Final

Resumen (ordenado por promedio ponderado - media de filas):
continent  mean_minutes_by_rows  mean_minutes_by_country_mean
     Asia                833.30                        833.30
  Oce

In [8]:
fig.write_html("minutes_played_by_continent.html")
print("Chart saved as minutes_played_by_continent.html")

Chart saved as minutes_played_by_continent.html
